In [ ]:
%config Completer.use_jedi = False

## Build the application

In [ ]:
from vespa.package import Document, Field

context_document = Document(
    fields=[
        Field(name="questions", type="array<int>", indexing=["summary", "attribute"]),
        Field(name="dataset", type="string", indexing=["summary", "attribute"]),
        Field(name="context_id", type="int", indexing=["summary", "attribute"]),        
        Field(name="text", type="string", indexing=["summary", "index"], index="enable-bm25"),                
    ]
)

In [ ]:
from vespa.package import Schema, FieldSet, RankProfile

context_schema = Schema(
    name="context",
    document=context_document, 
    fieldsets=[FieldSet(name="default", fields=["text"])], 
    rank_profiles=[
        RankProfile(name="bm25", inherits="default", first_phase="bm25(text)"), 
        RankProfile(name="nativeRank", inherits="default", first_phase="nativeRank(text)")]
)

In [ ]:
from vespa.package import HNSW

sentence_document = Document(
    inherits="context", 
    fields=[
        Field(
            name="sentence_embedding", 
            type="tensor<float>(x[512])", 
            indexing=["attribute", "index"], 
            ann=HNSW(
                distance_metric="euclidean", 
                max_links_per_node=16, 
                neighbors_to_explore_at_insert=500
            )
        )
    ]
)

In [ ]:
sentence_schema = Schema(
    name="sentence", 
    document=sentence_document, 
    fieldsets=[FieldSet(name="default", fields=["text"])], 
    rank_profiles=[
        RankProfile(
            name="semantic-similarity", 
            inherits="default", 
            first_phase="closeness(sentence_embedding)"
        ),
        RankProfile(
            name="bm25", 
            inherits="default", 
            first_phase="bm25(text)"
        ),
        RankProfile(
            name="bm25-semantic-similarity", 
            inherits="default", 
            first_phase="bm25(text) + closeness(sentence_embedding)"
        )
    ]
)

In [ ]:
from vespa.package import ApplicationPackage, QueryProfile, QueryProfileType, QueryTypeField

app_package = ApplicationPackage(
    name="qa", 
    schema=[context_schema, sentence_schema], 
    query_profile=QueryProfile(),
    query_profile_type=QueryProfileType(
        fields=[
            QueryTypeField(
                name="ranking.features.query(query_embedding)", 
                type="tensor<float>(x[512])"
            )
        ]
    )
)

In [ ]:
from vespa.package import VespaDocker

vespa_docker = VespaDocker(
    port=8081, 
    container_memory="8G", 
    disk_folder="/Users/tmartins/qa_app" # requires absolute path
)
app = vespa_docker.deploy(application_package=app_package)

## Feed sentence data

In [ ]:
from vespa.application import Vespa

app = Vespa(url = "http://localhost", port = 8081)

In [ ]:
import requests, json

context_data = json.loads(
    requests.get("https://data.vespa.oath.cloud/blog/qa/qa_squad_context_data.json").text
)

In [ ]:
import time
start_time = time.time()
for context in context_data:
    app.feed_data_point(schema="context", data_id=context["context_id"], fields=context)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
batch = [{"id": x["context_id"], "fields": x} for x in context_data]

In [ ]:
import time
start_time = time.time()
res = app.feed_batch(schema = "context", batch=batch)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
res

In [ ]:
from vespa.application import VespaAsync
import time

start_time = time.time()
async with VespaAsync(app) as async_app:
    await async_app.feed_batch(schema= "context", batch=batch)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import time
start_time = time.time()
res = app.feed_batch(schema = "context", batch=batch, asynchronous=True)
print("--- %s seconds ---" % (time.time() - start_time))
res

In [ ]:
import time
start_time = time.time()
res = await app.feed_batch(schema = "context", batch=batch, asynchronous=True)
print("--- %s seconds ---" % (time.time() - start_time))